# Regression Approach

This notebook is dedicated to modelling the problem as a regression problem. Different regression techniques are evaluated, and then the best-performing one is optimized in terms of hyper-parameters

In [1]:
# Change directory for cleaner path handling
%cd ..

C:\Users\georg\Documents\msc-project


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import tree, linear_model, dummy, kernel_ridge, gaussian_process
from sklearn.preprocessing import PolynomialFeatures,StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from src.evaluation import compare_models

## Feature selection

In [3]:
df = pd.read_csv("data/interim/tracks.csv")
targets = df['views']
features = [
    'danceability',
    'energy',
    'key',
    'loudness',
    'mode',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'time_signature',
    'duration_ms',
    'popularity'
]

data = df[features]

## Model comparison
Different regression models are compared with their default parameters to establish a baseline

In [19]:
models = [
    ("Baseline", dummy.DummyRegressor(strategy='mean')),
    ("Linear Regression", linear_model.LinearRegression()),
    ("Polynomial Regression", Pipeline([
        ('poly', PolynomialFeatures(degree=3)),
        ('linear', linear_model.LinearRegression())
    ])),
    ("Decision Tree", tree.DecisionTreeRegressor()),
    ("Kernel Ridge", kernel_ridge.KernelRidge(),
    ("Gaussian Process", gaussian_process.GaussianProcessRegressor()),
]
metrics = [
    'r2',
    'neg_mean_absolute_error',
    'neg_root_mean_squared_error',
]
compare_models(models, metrics, data, targets)

Evaluating Baseline
Evaluating Linear Regression
Evaluating Polynomial Regression
Evaluating Decision Tree
Evaluating Kernel Ridge
Evaluating Gaussian Process


,model,fit_time,score_time,test_r2,test_neg_mean_absolute_error,test_neg_root_mean_squared_error
0,Baseline,0.005755,0.001989,-0.070703,-6.882784e+06,-1.324836e+07
1,Linear Regression,0.006944,0.002281,-0.053032,-6.737619e+06,-1.312380e+07
2,Polynomial Regression,0.432908,0.012804,-7.952778,-8.266243e+06,-2.533421e+07
3,Decision Tree,0.113087,0.002678,-1.127108,-8.008613e+06,-1.806972e+07
4,Kernel Ridge,2.045796,0.040080,-0.126467,-5.664363e+06,-1.367353e+07
5,Gaussian Process,4.784020,0.812441,-6354.380109,-1.661785e+08,-8.411587e+08


## Model Optimization
The best performing model's hyperparameters will be optimized using Grid Search on the hyperparameter space

In [25]:
params = [
    {'kr__alpha': [0.1,1,10]},
    {'kr__kernel':['rbf','sigmoid','chi2'],'kr__gamma':[0.1,1,10]},
    {'kr__kernel':['polynomial'],'kr__degree':[2,3,4]}
]

model = Pipeline([
    ("std",StandardScaler()),
    ("kr",kernel_ridge.KernelRidge())
])

optimizer = GridSearchCV(
    model,
    param_grid=params,
    scoring='r2',
    n_jobs=3,
    verbose=4
)
optimizer.fit(data, targets)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


c:\users\georg\documents\msc-project\venv\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "c:\users\georg\documents\msc-project\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\users\georg\documents\msc-project\venv\lib\site-packages\sklearn\pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\users\georg\documents\msc-project\venv\lib\site-packages\sklearn\kernel_ridge.py", line 197, in f

GridSearchCV(estimator=Pipeline(steps=[('std', StandardScaler()),
                                       ('kr', KernelRidge())]),
             n_jobs=3,
             param_grid=[{'kr__alpha': [0.1, 1, 10]},
                         {'kr__gamma': [0.1, 1, 10],
                          'kr__kernel': ['rbf', 'sigmoid', 'chi2']},
                         {'kr__degree': [2, 3, 4],
                          'kr__kernel': ['polynomial']}],
             scoring='r2', verbose=4)